## TensorFlow Regression Analysis

### Step 1: Obtain Dataset

In [2]:
import numpy as np
import pandas as pd
import sqlite3
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Dense, Activation,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import layers
import tensorflow as tf

In [4]:
conn = sqlite3.connect("../db/stocks.db")
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
print(cursor.fetchall())

cmd = '''
SELECT g.Date AS Date,
        g."Adj Close" AS GrowthStock_AdjClose,
        (g."Adj Close" - LAG(g."Adj Close", 1) OVER (ORDER BY g.Date))/ 
                LAG(g."Adj Close", 1) OVER (ORDER BY g.Date) AS GrowthStock_PercentChange,
        
        v."Adj Close" AS ValueStock_AdjClose,
        (v."Adj Close" - LAG(v."Adj Close", 1) OVER (ORDER BY g.Date))/ 
                LAG(v."Adj Close", 1) OVER (ORDER BY g.Date) AS ValueStock_PercentChange,
        
        t."Adj Close" AS Treasury10yr_AdjClose,
        (t."Adj Close" - LAG(t."Adj Close", 1) OVER (ORDER BY g.Date))/ 
                LAG(t."Adj Close", 1) OVER (ORDER BY g.Date) AS Treasury10yr_PercentChange,
        (t."Adj Close" - LAG(t."Adj Close", 1) OVER (ORDER BY g.Date)) AS Treasury10yr_Diff,
        
        e.DEXCAUS AS exchange,
        (e.DEXCAUS - LAG(e.DEXCAUS, 1) OVER (ORDER BY g.Date))/ 
                LAG(e.DEXCAUS, 1) OVER (ORDER BY g.Date) AS exchange_PercentChange,
        (e.DEXCAUS - LAG(e.DEXCAUS, 1) OVER (ORDER BY g.Date)) AS exchange_Diff,
        
        i.T5YIE AS inflation5yr,
        (i.T5YIE - LAG(i.T5YIE, 1) OVER (ORDER BY g.Date))/ 
                LAG(i.T5YIE, 1) OVER (ORDER BY g.Date) AS inflation5yr_PercentChange,
        (i.T5YIE - LAG(i.T5YIE, 1) OVER (ORDER BY g.Date)) AS inflation5yr_Diff,
        
        c.CPIAUCSL_PC1 AS CPI,
        (c.CPIAUCSL_PC1 - LAG(c.CPIAUCSL_PC1, 1) OVER (ORDER BY g.Date))/ 
                LAG(c.CPIAUCSL_PC1, 1) OVER (ORDER BY g.Date) AS CPI_PercentChange,
        (c.CPIAUCSL_PC1 - LAG(c.CPIAUCSL_PC1, 1) OVER (ORDER BY g.Date)) AS CPI_Diff
        
FROM growth_stock g
JOIN value_stock v
ON g.Date = v.Date
JOIN treasury_10yr t
ON g.Date = t.Date
JOIN exchange e
ON g.Date = e.Date
JOIN inflation_5yr i
ON g.Date = i.Date
JOIN CPI c
ON g.Date = c.Date
'''

df = pd.read_sql_query(cmd, conn)
df.head()

[('growth_stock',), ('value_stock',), ('inflation_5yr',), ('exchange',), ('CPI',), ('leverage',), ('spending',), ('GDP',), ('treasury_10yr',)]


,Date,GrowthStock_AdjClose,GrowthStock_PercentChange,ValueStock_AdjClose,ValueStock_PercentChange,Treasury10yr_AdjClose,Treasury10yr_PercentChange,Treasury10yr_Diff,exchange,exchange_PercentChange,exchange_Diff,inflation5yr,inflation5yr_PercentChange,inflation5yr_Diff,CPI,CPI_PercentChange,CPI_Diff
0,2003-04-01,8.603225,NaN,26.337534,NaN,3.827,NaN,NaN,1.4729,NaN,NaN,1.64,NaN,NaN,2.17513,NaN,NaN
1,2003-05-01,9.085768,0.056089,28.422388,0.079159,3.856,0.007578,0.029,1.4191,-0.036527,-0.0538,1.30,-0.207317,-0.34,1.89415,-0.129178,-0.28098
2,2003-07-01,9.670133,0.064317,31.000845,0.090719,3.549,-0.079616,-0.307,1.3492,-0.049257,-0.0699,1.45,0.115385,0.15,2.05556,0.085215,0.16141
3,2003-08-01,9.606513,-0.006579,31.233463,0.007504,4.415,0.244012,0.866,1.3960,0.034687,0.0468,1.75,0.206897,0.30,2.21607,0.078086,0.16051
4,2003-10-01,10.052144,0.046388,32.274162,0.033320,3.932,-0.109400,-0.483,1.3481,-0.034312,-0.0479,1.84,0.051429,0.09,2.04194,-0.078576,-0.17413


In [5]:
df['Date'] = pd.to_datetime(df['Date'])
df = df.dropna()

### Step 2: Construct Function to Run the Model

In [ ]:
from sklearn.preprocessing import StandardScaler

def tfanalysis(target_stock, econ_variable, test_size, model):
    y = df[target_stock]
    X = df[econ_variable]

    # convert to np array
    X = np.array(X, dtype = np.float32)
    y = np.array(y, dtype = np.float32)
    
    # split train and test data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size)
    
    # normalize the dataset
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)